# Script to help user to define a configuration cell for ODC notebooks

*****

Geographical extent, time period, measurements, Coordinate Reference System (CRS) and resolution differ between the datasets available on the SDC.

This script will help you to create a configuration cell to load the data that you want, to be manually copy/pasted or loaded in ODC Jupyter notebook. You also find the overview of all datasets on the website: https://explorer.swissdatacube.org/products, and you can view the "measurements.csv" file within this folder for information on the available datasets.


In [1]:
# Import modules

# reload module before executing code
%load_ext autoreload
%autoreload 2

import pyproj
import numpy as np
import ipywidgets as widgets
import dask.distributed
from pystac_client import Client
import ast
from shapely.geometry import Polygon
from odc.stac import stac_load
import matplotlib.pyplot as plt
from odc import loader
import odc
from pyproj import Proj, transform
import pandas as pd
from sdc_utilities import *


# silence warning (not recommended during development)
import warnings
warnings.filterwarnings("ignore")

In [2]:
# Connect to server
client = dask.distributed.Client()

# This connects to the server and downloads a "catalog" tha contains some metadata
# catalog = Client.open("https://explorer.swissdatacube.org/stac")
catalog = Client.open("https://explorer.digitalearth.africa/stac")

# Search the catalog for all available products
collections = catalog.get_collections()

# List the collections (products) available in the catalog
product_names = [collection.id for collection in collections]

descriptions = []
# Get the descriptions and print a full list
for pi in product_names:
    collection = catalog.get_collection(pi)
    descriptions.append(collection.description)
    # print(f"{collection.id} - {collection.description}")

df = pd.DataFrame({'products': product_names, 'description': descriptions})
print(df)


                   products                                        description
0        alos_palsar_mosaic  ALOS/PALSAR and ALOS-2/PALSAR-2 annual mosaic ...
1             cci_landcover           ESA Climate Change Initiative Land Cover
2            cgls_landcover  Copernicus Global Land Service, Land Use/Land ...
3     cgls_lwq100_2019_2024  Copernicus Global Land Service – Lake Water Qu...
4      cgls_lwq100_2024_nrt  Copernicus Global Land Service – Lake Water Qu...
..                      ...                                                ...
95  wofs_ls_summary_alltime   Water Observations from Space Alltime Statistics
96   wofs_ls_summary_annual    Water Observations from Space Annual Statistics
97                 wsf_2015  The World Settlement Footprint WSF 2015 versio...
98                 wsf_2019  The World Settlement Footprint (WSF) 2019 is a...
99            wsf_evolution  The World Settlement Footprint (WSF) Evolution...

[100 rows x 2 columns]


# TESTS

In [37]:
# Connect to server
client = dask.distributed.Client()

# This connects to the server and downloads a "catalog" tha contains some metadata
# catalog = Client.open("https://explorer.swissdatacube.org/stac")
catalog = Client.open("https://explorer.digitalearth.africa/stac")

# Search the catalog for all available products
collections = catalog.get_collections()

# List the collections (products) available in the catalog
product_names = [collection.id for collection in collections]


In [74]:
root_catalog = catalog

In [75]:
root_catalog.description

'Configure stac endpoint information in your Explorer `settings.env.py` file'

In [76]:
items = list(root_catalog.get_all_items())

KeyboardInterrupt: 

In [73]:
root_catalog.get_child("ndvi_anomaly")

<CollectionClient id=ndvi_anomaly>

In [45]:
collections = list(catalog.get_collections())

In [50]:
print(collections[72])

<CollectionClient id=ndvi_anomaly>


In [61]:
collections[72].links
child_links = [l for l in coll.links if l.rel == "child"]
child_links
child = Client.open(child_links[0].target)

In [66]:
child_links[0].target

'https://explorer.digitalearth.africa/stac/catalogs/crop_mask/2019-1'

In [69]:
child
type(child)
# items = list(child.get_items())

pystac_client.client.Client

In [56]:

records = []

for coll in collections[72]:
    coll_id = coll#.id

    # Get all child links (e.g. yearly/monthly catalogs)
    child_links = [l for l in coll.links if l.rel == "child"]
    if not child_links:
        continue

    # Open first child catalog
    child = Client.open(child_links[0].target)

#     # Get first item (e.g. one tile of data)
#     items = list(child.get_items())
#     if not items:
#         continue

#     item = items[0]

#     # --- Extract projection info ---
#     epsg = item.properties.get("proj:epsg")
#     transform = item.properties.get("proj:transform")
#     shape = item.properties.get("proj:shape")
#     bbox = item.bbox

#     # Derive resolution from affine transform
#     if transform:
#         try:
#             aff = Affine(*transform)
#             xres, yres = abs(aff.a), abs(aff.e)
#         except Exception:
#             xres, yres = None, None
#     else:
#         xres, yres = None, None

#     records.append({
#         "product": coll_id,
#         "epsg": epsg,
#         "xres": xres,
#         "yres": yres,
#         "shape": shape,
#         "bbox": bbox
#     })

# # Combine into a dataframe
# df_meta = pd.DataFrame(records)
# pd.set_option("display.max_columns", None)
# print(df_meta)

TypeError: 'CollectionClient' object is not iterable

In [52]:
for coll in collections:
    print(f"\nCollection ID: {coll.id}")
    print(f"Description: {coll.description}")

    # Correct way to access summaries
    epsg = coll.summaries.get_list("proj:epsg")
    resolution = coll.summaries.get_list("proj:resolution")
    transform = coll.summaries.get_list("proj:transform")

    print("EPSG:", epsg)
    print("Resolution:", resolution)
    print("Transform:", transform)
    print("Bounding box:", coll.extent.spatial.bboxes)


Collection ID: alos_palsar_mosaic
Description: ALOS/PALSAR and ALOS-2/PALSAR-2 annual mosaic tiles generated for use in the Data Cube - 25m pixel spacing, WGS84. These tiles are derived from the orignal JAXA mosaics with conversion to GeoTIFF.
EPSG: None
Resolution: None
Transform: None
Bounding box: [[-20.000000000000004, -35.019721543030116, 55.000000000000014, 39.99999999999999]]

Collection ID: cci_landcover
Description: ESA Climate Change Initiative Land Cover
EPSG: None
Resolution: None
Transform: None
Bounding box: [[-26.361111111111054, -54.346951994584344, 64.50000000000004, 38.35000000000006]]

Collection ID: cgls_landcover
Description: Copernicus Global Land Service, Land Use/Land Cover at 100 m
EPSG: None
Resolution: None
Transform: None
Bounding box: [[-26.36011904761957, -54.34760583154707, 64.49999999999922, 38.35019841269862]]

Collection ID: cgls_lwq100_2019_2024
Description: Copernicus Global Land Service – Lake Water Quality 2019-2024 (raster 100 m), global, 10-dail

In [51]:
for coll in collections:
    print(f"\n{coll.id}")
    print("EPSG:", coll.summaries.get("proj:epsg"))
    print("Resolution:", coll.summaries.get("proj:shape"))
    print("BBox:", coll.extent.spatial.bboxes)


alos_palsar_mosaic


AttributeError: 'Summaries' object has no attribute 'get'

In [42]:
for coll in collections:
    print(coll.links)
    # child_links = [link for link in coll.links if link.rel == "child"]
    # print(child_links)

In [40]:
child_links

NameError: name 'child_links' is not defined

In [35]:
child_links = [link for link in coll.links if link.rel == "child"]

In [30]:
# import pandas as pd

# data = []

# for pi in product_names:
#     collection = catalog.get_collection(pi)
#     for coll in collections:
#         print(coll.id)
#         name = coll.id
#         child_links = [link for link in coll.links if link.rel == "child"]
#         if not child_links:
#             continue
    
#         # open first child (to avoid huge loops)
#         child = catalog.open(child_links[0].target)
#         items = list(child.get_items())
#         if not items:
#             continue
    
#         item = items[0]
    
#         epsg = item.properties.get("proj:epsg")
#         bbox = item.bbox
#         shape = item.properties.get("proj:shape")
#         transform = item.properties.get("proj:transform")
    
#         data.append({
#             "product": name,
#             "epsg": epsg,
#             "bbox": bbox,
#             "shape": shape,
#             "transform": transform
#         })
    
# df_meta = pd.DataFrame(data)
# print(df_meta)

In [27]:
# collection
collection = catalog.get_collection("ndvi_anomaly")
collection.links
# for c in collection:
#     c.child

[<Link rel=root target=<Client id=DEAfrica_data>>,
 <Link rel=self target=https://explorer.digitalearth.africa/stac/collections/ndvi_anomaly>,
 <Link rel=items target=https://explorer.digitalearth.africa/stac/collections/ndvi_anomaly/items>,
 <Link rel=http://www.opengis.net/def/rel/ogc/1.0/queryables target=https://explorer.digitalearth.africa/stac/collections/ndvi_anomaly/queryables>,
 <Link rel=child target=https://explorer.digitalearth.africa/stac/catalogs/ndvi_anomaly/2017-1>,
 <Link rel=child target=https://explorer.digitalearth.africa/stac/catalogs/ndvi_anomaly/2017-2>,
 <Link rel=child target=https://explorer.digitalearth.africa/stac/catalogs/ndvi_anomaly/2017-3>,
 <Link rel=child target=https://explorer.digitalearth.africa/stac/catalogs/ndvi_anomaly/2017-4>,
 <Link rel=child target=https://explorer.digitalearth.africa/stac/catalogs/ndvi_anomaly/2017-5>,
 <Link rel=child target=https://explorer.digitalearth.africa/stac/catalogs/ndvi_anomaly/2017-6>,
 <Link rel=child target=http

In [15]:
# # The full list:
# for row_i in range(df.shape[0]):
#     print(df.iloc[row_i,0] + ' --- ' + df.iloc[row_i,1])

In [3]:
# Allow user to select an available product prioritizing ingested (if any)over indexed

# Select the product
product_sel = widgets.RadioButtons(options=product_names, disabled=False)
display(widgets.Label('Select a product and run the next cell: '), product_sel)

Label(value='Select a product and run the next cell: ')

RadioButtons(options=('alos_palsar_mosaic', 'cci_landcover', 'cgls_landcover', 'cgls_lwq100_2019_2024', 'cgls_…

In [14]:
# somewhere the function str is getting assigned as new variable. not sure where. just reset it.
if not isinstance(str, type) or str.__name__ != 'str':
    # Restore the built-in str function
    del str

ds_selected = product_sel.value
df = pd.read_csv("measurements.csv")
dfs = df.loc[df['product']==ds_selected]
resolution = (-1*dfs['resolution'].min(),dfs['resolution'].min())

no_dates_str = 'No date infromation available. Visit https://explorer.swissdatacube.org/products for more information. You can leave the date also out for now.'
try:
    date_start = pd.to_datetime(dfs['time_start']).min()
except: 
    print(no_dates_str)
try:
    date_end = pd.to_datetime(dfs['time_end']).max()
except: 
    print(no_dates_str)

if date_start and date_end:
    time = (date_start.strftime('%Y-%m-%d'), date_end.strftime('%Y-%m-%d'))
# measurements_aliases = ['QA_PIXEL', 'blue', 'green', 'red', 'nir', 'swir_1', 'swir_2']
out = {}
aliases_ = {}
for r in range(len(dfs.index)):
    # r['measurement']
    _alias = dfs.iloc[r,:]['aliases']
    _alias = ast.literal_eval(_alias)
    _meas = dfs.iloc[r,:]['measurement']
    out[r] = _meas + ': ' + str(_alias)
    aliases_[r] = _alias[-1]

df = pd.DataFrame(list(out.values()), columns=['Bands'])

measur_sel = widgets.SelectMultiple(options=sorted(list(df['Bands'])),disabled=False)
display(widgets.Label('Select measurements (displayed with their aliases) and run the next cell: '), measur_sel)


ValueError: NaTType does not support strftime

In [5]:
# Convert selection to measurements list and message 

measur_list = list(measur_sel.value)

measur_sel = []
alias_sel = []
msg = '# to make your live easier you can manually replace the measurements variable by \n' \
      '# one of their alias:\n'
# cnt = 0
for m in measur_list:
    str_1 = m.split(': ')[0]
    str_2 = ast.literal_eval(m.split(': ')[1])[-1]
    measur_sel.append(str_1)
    if str_1 == "QA_PIXEL" or str_1 == "SCL":
        alias_sel.append(str_1)
    else:
        alias_sel.append(str_2)
    # cnt +=1

print(f"Using the following measurements {measur_sel}")
print(f"Corresponding to the aliases     {alias_sel}")

Using the following measurements ['SR_B2', 'SR_B3']
Corresponding to the aliases     ['green', 'red']


## Defining spatial extent

Open the website https://geojson.io/ and make your selection using the "rectangle" selection tool. The coordinates pop up on the right-hand side. Select the min and max values for the latitude and longitude. In the example this would be:

```
longitude =  (7.127, 7.199) 
latitude =  (46.773, 46.816)
```

Choose a <span style="color:red; font-size: 20px">***very small area*** </span>to test your workflow first, like in the example below.

Once you identified your bounding box (coordinates), add this into the cell below.

![geojson.io example](https://www.dropbox.com/scl/fi/9b93k2tsmk9u3nw2e3n0l/geojson.jpeg?rlkey=k5o3axx60eddd5nv58dx50bij&dl=1)

In [6]:
product = ds_selected
measurements = measur_sel

# Only change the extent in the following two lines :
longitude =  (7.127, 7.199) 
latitude =  (46.773, 46.816)
#  --------------------------------------------------

crs = 'epsg:4326'  # This is lat/lon geographic coordinates

# It makes sense to use a projected coordinates system for analysis related to areas and at high latitudes (Switzerland already above 45 degrees North)
output_crs = 'epsg:2056'   # (https://epsg.io/2056)

# You can define the final resolution. The original resolution is already stored in the variable "resolution". You can check it by making a new cell above and calling that variable
# resolution = -100.0, 100.0

In [7]:
# Select time period

start_date = widgets.DatePicker(description='Start date',
                                value = date_start.date(),
                                disabled=False)
end_date = widgets.DatePicker(description='End date',
                              value = date_end.date(),
                              disabled=False)
display(widgets.Label('IF REQUIRED define time period (cannot be outside of the initial displayed time) and run the next cell:'),
        widgets.HBox([start_date, end_date]))

Label(value='IF REQUIRED define time period (cannot be outside of the initial displayed time) and run the next…

In [8]:
# Check defined time period

assert start_date.value >= date_start.date(), \
       'Start date cannot be defined before {}'.format(date_start.date())
assert end_date.value <= date_end.date(), \
       'End date cannot be defined after {}'.format(date_end.date())
assert start_date.value <= end_date.value, \
       'End date is defined before start date'

# end_date = end_date.value + timedelta(days=1) # end_date is not inclusive !

print('Time period is OK')

Time period is OK


In [9]:
##### Resume configuration parameters in a format ready to be copy/pasted to a new cell,
# and in a txt file to be loaded with the '%load config_cell.txt' magic.

str = f'''# Configuration

product = '{product_sel.value}'
measurements = {measur_sel}
aliases = {alias_sel}  # you can also provide only the aliases and get the measurements with:
# measurements, aliases = get_alias_band(aliases)
{msg}
longitude = ({longitude[0]}, {longitude[1]})
latitude = ({latitude[0]}, {latitude[1]})
crs = 'epsg:4326'

time = ('{start_date.value.strftime('%Y-%m-%d')}', '{end_date.value.strftime('%Y-%m-%d')}')
# the following date formats are also valid:
# time = ('2000-01-01', '2001-12-31')
# time=('2000-01', '2001-12')
# time=('2000', '2001')

# You can use an UTM zone according to the DataCube System.
# We prefer not to use this, instead specifying SwissGrid (epsg:2056).
# output_crs = 'epsg:2056'

output_crs = '{output_crs}'
resolution = {resolution[0]}, {resolution[1]}

# These are the pixel classifications for Sentinel (SCL) and Landsat (QA_PIXEL); 
# you can use values to mask out values that belong to certain classes

###################################
# SCL categories:                 #
#   0 - no data                   #
#   1 - saturated or defective    #
#   2 - dark area pixels          #
#   3 - cloud_shadows             #
#   4 * vegetation                #
#   5 * not vegetated             #
#   6 * water                     #
#   7 * unclassified              #
#   8 - cloud medium probability  #
#   9 - cloud high probability    #
#  10 - thin cirrus               #
#  11 * snow                      #
###################################

# Check for more detailed information: 
# - Landsat 8/9 (OLI/TIRS), Page 19:
# https://d9-wret.s3.us-west-2.amazonaws.com/assets/palladium/production/s3fs-public/media/files/LSDS-1619_Landsat8-9-Collection2-Level2-Science-Product-Guide-v6.pdf
# - Landsat 7 (ETM+), Page 15:
# https://d9-wret.s3.us-west-2.amazonaws.com/assets/palladium/production/s3fs-public/media/files/LSDS-1337_Landsat7ETM-C2-L2-DFCB-v6.pdf
# - Landsat 4,5 (TM), Page 18:
# https://d9-wret.s3.us-west-2.amazonaws.com/assets/palladium/production/s3fs-public/atoms/files/LSDS-1415_Landsat4-5-TM-C2-L1-DFCB-v3.pdf

#############################################
# QA_PIXEL BITS : CATEGORIES                #
#    0 : Fill                               #
#    1 : Clear                              #
#    2 : Water                              #
#    3 : Cloud shadow                       #
#    4 : Snow                               #
#    5 : Cloud                              #
#   10 : Terrain occlusion (Landsat 8 only) #
#############################################

chunks = {{"x": 2048, "y": 2048, "time": 1}}  # 2048 values are OK with ~21Gb memory available
'''
print(str)
with open('config_cell.txt', 'w') as text_file:
    print(str, file=text_file)

# Configuration

product = 'landsat_etm_c2_l2'
measurements = ['SR_B2', 'SR_B3']
aliases = ['green', 'red']  # you can also provide only the aliases and get the measurements with:
# measurements, aliases = get_alias_band(aliases)
# to make your live easier you can manually replace the measurements variable by 
# one of their alias:

longitude = (7.127, 7.199)
latitude = (46.773, 46.816)
crs = 'epsg:4326'

time = ('1999-06-01', '2023-10-01')
# the following date formats are also valid:
# time = ('2000-01-01', '2001-12-31')
# time=('2000-01', '2001-12')
# time=('2000', '2001')

# You can use an UTM zone according to the DataCube System.
# We prefer not to use this, instead specifying SwissGrid (epsg:2056).
# output_crs = 'epsg:2056'

output_crs = 'epsg:2056'
resolution = -30.0, 30.0

# These are the pixel classifications for Sentinel (SCL) and Landsat (QA_PIXEL); 
# you can use values to mask out values that belong to certain classes

###################################
# SCL categories: